# Rapport de projet final: Débordement d'égouts
## Équipe 9 - A19
#### _Noboru Yoshida - Mehdi Chaid - Mathieu Giroux-Huppé - Maxime Gosselin_
<br>
20 Décembre 2019

## Table des matières

+ [1. Introduction](#1.)
    + [1.1. Objectifs](#1.1.)  
    + [1.2. Méthode de travail](#1.2.)  
<br>
+ [2. Analyse exploratoire](#2.)
    + [2.1. Motivation](#2.1.)  
    + [2.2. Importation des données](#2.2.)  
    + [2.3. Fonctions utilisées](#2.3.)
    + [2.4. Analyse des données d'ouvrages](#2.4.)  
        + [2.4.1. Données discrètes](#2.4.1.)
        + [2.4.2. Données de Trop-Plein](#2.4.2.)
        + [2.4.3. Données d'Emissaire](#2.4.3.)
    + [2.5. Analyse des données de précipitations](#2.5.)  
    + [2.6. Analyse des données de surverses](#2.6.)  
<br>
+ [3. Traitement des données](#3.)
    + [3.1. Motivation](#3.1.)  
    + [3.2. Fonctions utilisées](#2.2.)  
    + [3.3. Ajout de variables explicatives](#3.3.)
    + [3.4. Standardisation et OneHot](#3.4.)  
    + [3.5. Séparation en ensembles de tests et validation](#3.5.)  
<br>
+ [4. Sélection de modèles](#4.)
    + [4.1. Motivation](#4.1)
    + [4.2. Choix des modèles](#4.2.)
        + [4.2.1. Arbres de décisions](#4.2.1.)
        + [4.2.2. Régression logistique](#4.2.2.)
        + [4.2.3. Classification bayésienne naive](#4.2.3.)
    + [4.3. Arbres de décision et forêt aléatoire](#4.3.)  
    + [4.4. Régression logistique](#4.4.)  
    + [4.5. Classification bayésienne naive](#4.5.)  
    + [4.6. Ensemble de modèles](#4.6.)  
<br>
+ [5. Retour et conclusion](#5.)
    + [5.1. Retour sur les résultats](#5.1.)  
    + [5.2. Améliorations possibles](#5.2.)  
    + [5.3. Retour sur le travail en équipe](#5.3.)  
    + [5.4. Conclusion](#5.4.)  
<br>

<a id="1."></a>

## 1. Introduction

<a id="1.1."></a>
### 1.1. Objectifs

Ce rapport vise à fournir une prédiction sur les surverses dans plusieurs ouvrages sur l'île de Montréal. Il en existe 170 répartis sur tout le bord de l'île. Dans ce travail, nous nous attarderons sur la prédiction de seulement 5 d'entre-eux, soit les suivants :
- 3260-01D dans Rivière-des-Prairies
- 3350-07D dans Ahunstic
- 4240-01D dans Pointe-aux-Trembles
- 4350-01D dans le Vieux-Montréal
- 4380-01D dans Verdun

Nous avons à notre disposition 3 jeux de données qui nous aiderons à trouver une relation entre la quantité de pluie tombé et les surverses de certain ouvrages. Nous devrons alors entraîner un modèle sur les données de 2013 à 2018, puis prédire sur certaines dates de 2019. Il n'y pas de restrictions au niveau des techniques à utilisés pour le type de modèle à utiliser. Nous avons exploré quelques techniques différentes que nous allons détaillé plus en détail dans la [section 5](#model-selection).

<a id="1.2."></a>
### 1.2. Méthode de travail

La méthode de travail employée par notre équipe fut d'abord de mettre tout le monde sur une exploration initiale des données, afin de comprendre en équipe la signification générale de chaque variable explicative fournie, puis d'effectuer une séparation des tâches. L'exploration des données en équipe a aussi été utile pour déterminer les modèles qui nous semblaient les plus intéressants et appropriés selon le contexte.

<br>
Par la suite, une personne était en charge d'effectuer une exploration plus poussée des données tandis que chaque autre membre de l'équipe travaillait sur l'analyse puis l'application d'un modèle différent. Étant 4 membres dans l'équipe, et ayant repéré 3 modèles intéressants, cette séparation fut facile à faire et nous arrangeait.

<br>

Enfin, une fois que nous avions obtenu un prototype fonctionnel pour chacun des modèles, nous nous réunissions en équipe afin d'analyser les résultats et d'améliorer les modèles.

<a id="2."></a>
## 2. Analyse exploratoire

<a id="2.1."></a>
### 2.1. Motivation

L'analyse exploratoire est une très importante pour tout exercice en sciences de données. Elle permet de déterminer, entres autres:

- Quelles variables vont jouer un rôle important pour la prédiction.
- À quel point les variables sont corrélées entre elles.
- Si une régularisation est nécessaire pour certaines variables.
- Comment améliorer le temps d'entrainement et les prédictions.

Pour ce faire, nous allons analyser chacune des variables explicatives en utilisant des graphes et des statistiques afin d'appuyer nos résultats et de guider notre prise de décision.

<a id="2.2."></a>
### 2.2. Importation des données

Installons d'abord les libraires utilisées pour l'analyse exploratoire

In [ ]:
using Pkg;
Pkg.add("CSV");
Pkg.add("Random");
Pkg.add("DataStructures");
Pkg.add("BenchmarkTools");
Pkg.add("DataFrames");
Pkg.add("Statistics");
Pkg.add("Dates");
Pkg.add("Gadfly");
Pkg.add("IterTools");

Importons maintenant ces données dans le rapport

In [3]:
using CSV, DataFrames, DataStructures, BenchmarkTools, Statistics, Dates, Gadfly, Random, IterTools;

<a id="2.3."></a>
### 2.3. Fonctions utilisées